<br><font face="Times New Roman" size=5><div dir=ltr align=center>
<font color=blue size=8>
    Introduction to Machine Learning <br>
<font color=red size=5>
    Sharif University of Technology - Computer Engineering Department <br>
    Fall 2022<br> <br>
<font color=black size=6>
    Homework 2: Practical - Decision Tree   
<font color=black size=4>
    Hamidreza Yaghoubi 
    
<br><br>
<font size=4>
In this homework, we are going to implement the Classification Decision Tree. Keep in mind to complete all of the following questions and write your own codes in the TODO cells.

<font face="Times New Roman" size=4><div dir=ltr>
# Problem 2: Classification Decision Tree (100 points)
We will implement a Classification Decision Tree from scratch in the following problem. Then we will use our model to predict malignant and benign breast cancer. For this purpose, we will use the breast_cancer.csv dataset which you can find more details about it <a href="https://www.kaggle.com/datasets/merishnasuwal/breast-cancer-prediction-dataset"><font face="Roboto">here</font></a>.

In [1]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from math import log
import random
import numpy as np

<font face="Times New Roman" size=4><div dir=ltr>
## Classification Decision Tree Class (60 points)
In this section, you only need to fill TODO parts. You can find the logic and formula in both course slides and the web, but fill it out on your own. 

In [2]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf(self):
        if self.value is not None:
            return True
        return False

In [3]:
class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def is_splitting_finished(self, depth, X, y):
        #TODO
        n_samples, n_features = X.shape
        n_class_labels = len(np.unique(y))
        if depth >= self.max_depth or n_class_labels == 1 or n_samples < self.min_samples_split:
            return True
        return False

    def split(self, X, y, feature, threshold):
        left_idx = np.argwhere(X[:, feature] <= threshold).flatten()
        right_idx = np.argwhere(X[:, feature] > threshold).flatten()
        X_left = X[left_idx]
        y_left = y[left_idx]
        X_right = X[right_idx]
        y_right = y[right_idx]

        return X_left, X_right, y_left, y_right

    def entropy(self, y):
        proportions = np.bincount(y) / len(y)
        ent = -np.sum([p * np.log2(p) for p in proportions if p > 0])
        return ent
        #TODO


    def information_gain(self, X, y, feature, threshold):
        parent_loss = self.entropy(y)
        X_left, X_right, y_left, y_right = self.split(X, y, feature, threshold)
        n, n_left, n_right = len(X), len(X_left), len(X_right)
        
        if n_left == 0 or n_right == 0: 
            return 0
        
        child_loss = (n_left / n) * self.entropy(y_left) + (n_right / n) * self.entropy(y_right)
        return parent_loss - child_loss
        #TODO

    def best_split(self, X, y):
        split = {'score':- 1, 'feat': None, 'thresh': None}
        for i in range(len(X[0])):
            X_feature = X[:, i]
            thresholds = np.unique(X_feature)
            for thresh in thresholds:
                score = self.information_gain(X, y, i, thresh)
                if score > split['score']:
                    split['score'] = score
                    split['feat'] = i
                    split['thresh'] = thresh

        return split['feat'], split['thresh']
        #TODO

    def build_tree(self, X, y, depth=0):
        if self.is_splitting_finished(depth, X, y):
            return None

        best_feature, best_threshold = self.best_split(X, y)
        X_left, X_right, y_left, y_right = self.split(X, y, best_feature, best_threshold)

        left_node = self.build_tree(X_left, y_left, depth=depth + 1)
        right_node = self.build_tree(X_right, y_right, depth=depth + 1)

        value = None
        if left_node is None or right_node is None:
            true_value = len(y[y == 1])
            false_value = len(y[y == 0])
            if true_value >= false_value:
                value = 1
            else:
                value = 0

        return Node(feature=best_feature, threshold=best_threshold, left=left_node, right=right_node, value=value)

    def fit(self, X, y):
        self.root = self.build_tree(X, y)
    
    def traverse_tree(self, x, node):
        if node.is_leaf():
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self.traverse_tree(x, node.left)
        return self.traverse_tree(x, node.right)

    def predict(self, X):
        predictions = [self.traverse_tree(x, self.root) for x in X]
        return np.array(predictions)

<font face="Times New Roman" size=4><div dir=ltr>
## Data Prepration (20 points)
In this section, you must perform a good EDA for data. Then split it into train and validation data. We will then use the validation data to find the best model hyperparameters.  

In [4]:
breast_cancer_pdf = pd.read_csv("breast_cancer.csv")

In [5]:
# TODO

In [6]:
data = datasets.load_breast_cancer()
print(breast_cancer_pdf.shape)
X, y = breast_cancer_pdf.iloc[:, 0:5].to_numpy(), breast_cancer_pdf["diagnosis"].to_numpy()

x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)

(569, 6)


<font face="Times New Roman" size=4><div dir=ltr>
## Training And Tuning Hyperparameters (20 points)
In this section, you only need to find the best hyperparameters for your model. You can test different values and permutations of hyperparameters by adding them to the lists below. Your model must have at least accuracy=0.85 on validation data.

In [7]:
max_depths = [1, 2, 5, 20, 30]
min_samples_splits = [30, 50, 70, 100, 130]

In [8]:
best_max_depth = None
best_min_samples_split = None
best_accuracy = 0
best_model = None
for max_depth in max_depths:
    for min_samples_split in min_samples_splits:
        clf = DecisionTree(max_depth, min_samples_split)
        clf.fit(x_train, y_train)
        y_val_pred = clf.predict(x_val)
        y_train_pred = clf.predict(x_train)
        accuracy = accuracy_score(y_val_pred, y_val)
        train_accuracy = accuracy_score(y_train_pred, y_train)
        print(f"accuracy of training set for [min_samples_splits={min_samples_split}-max_depths={max_depth}] ={train_accuracy}")
        print(f"accuracy of validation set for [min_samples_splits={min_samples_split}-max_depths={max_depth}] ={accuracy}")
        print("------------------------------------------------")
        if accuracy >= best_accuracy:
            best_accuracy = accuracy
            best_max_depth = max_depth
            best_min_samples_split = min_samples_split
            best_model = clf

accuracy of training set for [min_samples_splits=30-max_depths=1] =0.6285714285714286
accuracy of validation set for [min_samples_splits=30-max_depths=1] =0.6228070175438597
------------------------------------------------
accuracy of training set for [min_samples_splits=50-max_depths=1] =0.6285714285714286
accuracy of validation set for [min_samples_splits=50-max_depths=1] =0.6228070175438597
------------------------------------------------
accuracy of training set for [min_samples_splits=70-max_depths=1] =0.6285714285714286
accuracy of validation set for [min_samples_splits=70-max_depths=1] =0.6228070175438597
------------------------------------------------
accuracy of training set for [min_samples_splits=100-max_depths=1] =0.6285714285714286
accuracy of validation set for [min_samples_splits=100-max_depths=1] =0.6228070175438597
------------------------------------------------
accuracy of training set for [min_samples_splits=130-max_depths=1] =0.6285714285714286
accuracy of validat

In [9]:
print("accuracy of validation set for best model:", best_accuracy)
print("best max depth:", best_max_depth)
print("best min sample:", best_min_samples_split)

accuracy of validation set for best model: 0.9210526315789473
best max depth: 30
best min sample: 130
